In [1]:
# テキストデータの準備
import os
import json

# コーパスのディレクトリを設定
file_path = './projectnextnlp-chat-dialogue-corpus/json/rest1046/'
# ファイルの一覧を取得
file_dir = os.listdir(file_path)
# 正解ラベルとテキストデータを保持するリスト
label_text = []

# ファイルごとに対話データを整形する
for file in file_dir:
    # JSONファイルの読み込み
    r = open(file_path + file, 'r', encoding='utf-8')
    json_data = json.load(r)
        
    # 発話データ配列から発話テキストと破綻かどうかの正解データを抽出
    for turn in json_data['turns']:
        turn_index = turn['turn-index'] # turn-indexキー(対話のインデックス)
        speaker = turn['speaker']       # speakerキー("U"人間、"S"システム)
        utterance = turn['utterance']   # utteranceキー(発話テキスト)
        
        # 先頭行(システムの冒頭の発話)以外を処理
        if turn_index != 0:
            # 人間の発話（質問）のテキストを抽出
            if speaker == 'U':
                #u_text = ''
                u_text = utterance

            # システムの回答内容が破綻かどうかを抽出
            else:
                a = ''
                sys = turn['utterance'] # システムの発話（応答）を抽出
                t = turn['annotations'][0] # １つ目のアノテーションを抽出                
                a = t['breakdown']      # アノテーションのフラグを抽出
                if a == 'O':            # O（破綻していない）を0で置換
                    label = 0
                else:                   # O以外は破綻とし1で置換
                    label = 1
                # 正解ラベルをタブで区切って「人間の発話#システムの発話」を連結
                tmp1 = str(label) + '\t' + u_text + ' # ' +sys
                # タブで区切ってリストにする
                tmp2 = tmp1.split('\t')
                # 作成したリストを要素としてlabel_textに追加
                label_text.append(tmp2)

In [2]:
# 正解ラベルと発話データを出力
label_text

[['0', 'こんにちは # こんー'],
 ['0', '分からない # そっか'],
 ['0', '昼ごはんは何を食べましたか # ごはんはあったかいです'],
 ['0', 'へー # えー'],
 ['0', 'へー # えー'],
 ['1', 'それで夕食は何を食べるのですか # 夕食がいってないかもです'],
 ['1', '日本語として変です # 日本語は早いです'],
 ['0', 'ばいばい # ばいばーい'],
 ['0', 'さようなら # どうした'],
 ['0', 'それでそれで # ですよねー'],
 ['1', 'そうだね。でもまだ食べてないや。 # スイカで買い物できてるのですか？'],
 ['0', 'よくわからないよ # 買い物は女同士がいいですね'],
 ['1', 'そうかな？私は旦那さんと買い物するよ。 # 旦那が行きますか？'],
 ['0', '一緒にね # 一緒に'],
 ['0', 'そう # そうか'],
 ['1', 'そうです # やっぱり'],
 ['1', 'なんでわかるの？ # パリで使います'],
 ['1', '何を使うのかしら？ # パリをはじめます'],
 ['1', 'まずはフランス語を勉強してね！ # フランス語で歌われるのです'],
 ['0', 'なにをですか？ # フランス語を覚えるのです'],
 ['0', 'こんばんは。最近暑い日が続いていて、熱中症で倒れる人も多いみたいですね。 # 予防が大切ですね'],
 ['0', 'ええ、僕もこまめに水分補給をするようにしています。 # 予防は完璧ですね'],
 ['1', 'はい。あなたは何か熱中症対策をしていますか？ # 予防が肝心です'],
 ['1', 'そういえば、梅雨が明けましたね。 # 日頃から予防を心掛けるかもしれない'],
 ['0', '転ばぬ先の杖というやつですね。 # 予防は大切なんですよねー'],
 ['1', 'まったくですねー。 # ごめんなさい'],
 ['1', 'いえいえ、気にする必要はありませんよ。 # 予防は大切です'],
 ['0', '分かってはいても、ついつい怠ってしまうんですよね。 # 予防は大切ですね'],
 ['0', 'はい。子供は炎天下の中で遊びまわるので、大人が気を付けてあげな

In [3]:
# データのサイズを出力
len(label_text)

10460

In [4]:
# 形態素への分解と正解ラベルのリストの生成
from janome.tokenizer import Tokenizer # janomeのパッケージをインポート
import re                              # 正規表現ライブラリ

t = Tokenizer()                       # Tokenizerクラスのオブジェクトを生成
separation_tmp = []                   # 形態素を一時保存するリスト

# 形態素に分解
for row in label_text:
    # リストから発話テキストの部分を抽出して形態素解析を実行
    tokens = t.tokenize(row[1])
    # 形態素の見出しの部分を取得してseparation_tmpに追加
    separation_tmp.append(
        [token.surface for token in tokens if (
            not re.match('記号', token.part_of_speech)             # 記号を除外
            and (not re.match('助詞', token.part_of_speech))       # 助詞を除外
            and (not re.match('助動詞', token.part_of_speech))     # 助動詞を除外
            and (not re.match('動詞,非自立', token.part_of_speech))# 動詞,非自立を除外
            and (not re.match('名詞,非自立', token.part_of_speech))# 名詞,非自立を除外
            )
         ])
    # 空の要素があれば取り除く
    while separation_tmp.count('') > 0:
        separation_tmp.remove('')

# 正解ラベルをint型に変換してリストに格納
train_y_tmp = [int(label[0]) for label in label_text]

separation = [] # 形態素のリスト
train_y = []    # 正解ラベルのリスト

# separation_tmpの値が存在すれば、
# 形態素のリストと正解ラベルのリストを作成
for x, y in zip(separation_tmp, train_y_tmp):
    if x:
        separation.append(x)
        train_y.append(y)

In [5]:
separation

[['こんにちは', '#', 'こん', 'ー'],
 ['分から', '#', 'そっ'],
 ['昼', 'ごはん', '何', '食べ', '#', 'ごはん', 'あったかい'],
 ['へー', '#', 'えー'],
 ['へー', '#', 'えー'],
 ['それで', '夕食', '何', '食べる', '#', '夕食', 'いっ'],
 ['日本語', '変', '#', '日本語', '早い'],
 ['ばい', 'ばい', '#', 'ばい', 'ー', 'い'],
 ['さようなら', '#', 'どう', 'し'],
 ['それで', 'それ', '#'],
 ['そう', 'でも', 'まだ', '食べ', '#', 'スイカ', '買い物', 'でき'],
 ['よく', 'わから', '#', '買い物', '女', '同士', 'いい'],
 ['そう', '私', '旦那', 'さん', '買い物', 'する', '#', '旦那', '行き'],
 ['一緒', '#', '一緒'],
 ['そう', '#', 'そう'],
 ['そう', '#', 'やっぱり'],
 ['なんで', 'わかる', '#', 'パリ', '使い'],
 ['何', '使う', '#', 'パリ', 'はじめ'],
 ['まずは', 'フランス語', '勉強', 'し', '#', 'フランス語', '歌わ', 'れる'],
 ['なに', '#', 'フランス語', '覚える'],
 ['こんばんは', '最近', '暑い', '続い', '熱中', '症', '倒れる', '人', '多い', '#', '予防', '大切'],
 ['ええ', '僕', 'こまめ', '水分', '補給', 'する', 'し', '#', '予防', '完璧'],
 ['はい', 'あなた', '何', '熱中', '症', '対策', 'し', '#', '予防', '肝心'],
 ['そう', 'いえ', '梅雨', '明け', '#', '日頃', '予防', '心掛ける', 'しれ'],
 ['転ば', '先', '杖', 'やつ', '#', '予防', '大切'],
 ['まったく', '#', 'ごめんなさい'],
 ['いえいえ', '

In [6]:
# 正解ラベル
train_y

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,


In [7]:
# 単語の出現回数を記録して辞書を作成　
from collections import Counter        # カウント処理のためのライブラリ
import itertools                       # イテレーションのためのライブラリ
# {単語：出現回数}の辞書を作成
word_frequency = Counter(itertools.chain(* separation))

In [8]:
# 単語の出現回数を出力
word_frequency

Counter({'こんにちは': 212,
         '#': 10460,
         'こん': 20,
         'ー': 288,
         '分から': 16,
         'そっ': 12,
         '昼': 18,
         'ごはん': 22,
         '何': 576,
         '食べ': 346,
         'あったかい': 3,
         'へー': 15,
         'えー': 37,
         'それで': 10,
         '夕食': 36,
         '食べる': 112,
         'いっ': 23,
         '日本語': 18,
         '変': 6,
         '早い': 32,
         'ばい': 23,
         'い': 113,
         'さようなら': 9,
         'どう': 194,
         'し': 910,
         'それ': 228,
         'そう': 1060,
         'でも': 108,
         'まだ': 43,
         'スイカ': 572,
         '買い物': 89,
         'でき': 132,
         'よく': 176,
         'わから': 45,
         '女': 18,
         '同士': 10,
         'いい': 1409,
         '私': 378,
         '旦那': 15,
         'さん': 40,
         'する': 252,
         '行き': 443,
         '一緒': 182,
         'やっぱり': 36,
         'なんで': 45,
         'わかる': 14,
         'パリ': 10,
         '使い': 33,
         '使う': 25,
         'はじめ': 4,
         'まずは': 4

In [9]:
# 単語を頻度降順に並べ替え
word_list = []
# most_common()で出現回数順に要素を取得しword_listに追加
for w in word_frequency.most_common():
    word_list.append(w[0])
    
# 頻度順に並べた単語をキーに、1から始まる連番を値に設定
word_dic = {}
for i, word in enumerate(word_list, start=1):
    word_dic.update({word: i})

In [10]:
# 並べ替えの結果を出力
word_dic

{'#': 1,
 'いい': 2,
 '好き': 3,
 'そう': 4,
 'し': 5,
 '海': 6,
 '何': 7,
 'スイカ': 8,
 '行き': 9,
 'あり': 10,
 '退屈': 11,
 '私': 12,
 '食べ': 13,
 'お': 14,
 '気': 15,
 'そうですね': 16,
 '症': 17,
 '熱中': 18,
 'ー': 19,
 '良い': 20,
 'はい': 21,
 '人': 22,
 '楽しい': 23,
 '見': 24,
 'する': 25,
 'ない': 26,
 'うん': 27,
 'つけ': 28,
 '多い': 29,
 '大好き': 30,
 'それ': 31,
 'こんにちは': 32,
 '大丈夫': 33,
 'ある': 34,
 '最近': 35,
 '美味しい': 36,
 'どう': 37,
 'ありがとう': 38,
 '雨': 39,
 '行っ': 40,
 'さ': 41,
 '一緒': 42,
 'あなた': 43,
 '行く': 44,
 'どんな': 45,
 'よく': 46,
 '夏': 47,
 'どこ': 48,
 '暑い': 49,
 'れ': 50,
 '欲しい': 51,
 'なり': 52,
 '今日': 53,
 'しれ': 54,
 '思い': 55,
 '心': 56,
 '仕事': 57,
 'おいしい': 58,
 '有名': 59,
 '気持ち': 60,
 '趣味': 61,
 'でき': 62,
 '哲学': 63,
 '面白い': 64,
 '映画': 65,
 '朝': 66,
 'ゲーム': 67,
 'なっ': 68,
 '僕': 69,
 '一': 70,
 '大切': 71,
 '見る': 72,
 'い': 73,
 '食べる': 74,
 '旅行': 75,
 '天気': 76,
 '高い': 77,
 '大事': 78,
 'でも': 79,
 '予防': 80,
 '今': 81,
 '楽しみ': 82,
 '行か': 83,
 '嫌い': 84,
 'いや': 85,
 'こんばんは': 86,
 'プール': 87,
 '買い物': 88,
 '水分': 89,
 '必要': 90,
 'スポーツ': 9

In [11]:
# 辞書のサイズを出力
len(word_dic)

6755

In [12]:
# 単語を出現頻度の数値に置き替える
train_x = [[ word_dic[word] for word in sp] for sp in separation ]

In [13]:
# 結果の出力
train_x

[[32, 1, 448, 19],
 [557, 1, 729],
 [509, 405, 7, 13, 1, 405, 2320],
 [586, 1, 240],
 [586, 1, 240],
 [853, 250, 7, 74, 1, 250, 380],
 [510, 1309, 1, 510, 277],
 [381, 381, 1, 381, 19, 73],
 [944, 1, 37, 5],
 [853, 31, 1],
 [4, 79, 194, 13, 1, 8, 88, 62],
 [46, 183, 1, 88, 511, 854, 2],
 [4, 12, 587, 213, 88, 25, 1, 587, 9],
 [42, 1, 42],
 [4, 1, 4],
 [4, 1, 251],
 [184, 624, 1, 855, 271],
 [7, 356, 1, 855, 1844],
 [1845, 2321, 512, 5, 1, 2321, 1846, 120],
 [105, 1, 2321, 856],
 [86, 35, 49, 4093, 18, 17, 2322, 22, 29, 1, 80, 71],
 [302, 69, 857, 89, 222, 25, 5, 1, 80, 625],
 [21, 43, 7, 18, 17, 231, 5, 1, 80, 858],
 [4, 98, 423, 2323, 1, 859, 80, 1847, 54],
 [4094, 424, 4095, 347, 1, 80, 71],
 [1310, 1, 357],
 [241, 15, 25, 90, 10, 1, 80, 71],
 [788, 73, 2324, 4096, 1, 80, 71],
 [21, 358, 1848, 348, 4097, 588, 15, 151, 1, 1848, 232],
 [302, 303, 513, 2325, 2322, 22, 29, 4, 1, 303, 513, 290],
 [449,
  38,
  1,
  169,
  2928,
  2929,
  4098,
  4099,
  4100,
  4101,
  4102,
  4103,
  5,


In [14]:
# 単語の出現頻度の数値への置き替えと正解ラベルのOne-hot表現への変換
from keras.preprocessing import sequence
from keras.utils import np_utils
# 単語データの配列のサイズを合わせる
trainX = sequence.pad_sequences(train_x, maxlen=32, padding='post',value=0.0)
# 正解ラベルをOne-hot表現にする
trainY = np_utils.to_categorical(train_y, 2)

Using TensorFlow backend.


In [15]:
# 訓練データと正解ラベルのサイズとデータの表示
print(trainX.shape)
print(trainX)
print(trainY.shape)
print(trainY)

(10460, 32)
[[ 32   1 448 ...   0   0   0]
 [557   1 729 ...   0   0   0]
 [509 405   7 ...   0   0   0]
 ...
 [302   1  37 ...   0   0   0]
 [ 98 460   5 ...   0   0   0]
 [728 491  37 ...   0   0   0]]
(10460, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [16]:
# RNNの構築
from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam

# ニューラルネットワークの元になるSequentialオブジェクトを生成
model = Sequential()

## 入力層 ##
# 入力データのサイズは32
model.add(InputLayer(input_shape=(32,)))

## 中間層 ##
# 単語埋め込み層
model.add(
    Embedding(
        input_dim=len(word_dic) + 1, # 単語の総数に0のための1を加算
        output_dim=128               # 出力は中間層のユニット数と同数
    ))

# LSTMブロック（ユニット数＝128）×3段
model.add(LSTM(128, dropout=0.5, return_sequences=True))
model.add(LSTM(128, dropout=0.5, return_sequences=True))
model.add(LSTM(128, dropout=0.5, return_sequences=False))

## 出力層 ##
model.add(Dense(units=2,             # 出力層のニューロン数は2
                activation='softmax' # 活性化はシグモイド関数
               ))

# Squentialオブジェクをコンパイル
model.compile(
    loss='categorical_crossentropy', # 誤差関数はクロスエントロピー
    optimizer=Adam(),                # Adamオプティマイザー
    metrics=['accuracy']             # 学習評価として正解率を指定
    )

model.summary()                      # RNNのサマリー（概要）を出力

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           864768    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32, 128)           131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 32, 128)           131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 1,259,778
Trainable params: 1,259,778
Non-trainable params: 0
_________________________________________________________________


In [17]:
# 学習を行う
history = model.fit(trainX, trainY,        # 訓練データ、正解ラベル
                    batch_size=32,         # ミニバッチのサイズ
                    epochs=50,             # 学習回数
                    verbose=1,             # 学習の進捗状況を出力する
                    validation_split=0.2,  # 訓練データのうち、テストデータとして使用する割合
                    shuffle=True           # テストデータ抽出後にシャッフル
                    )

Train on 8368 samples, validate on 2092 samples
Epoch 1/50
8368/8368 [==============================] - 32s 4ms/step - loss: 0.6945 - acc: 0.5049 - val_loss: 0.6941 - val_acc: 0.4976
Epoch 2/50
8368/8368 [==============================] - 30s 4ms/step - loss: 0.6932 - acc: 0.5135 - val_loss: 0.6936 - val_acc: 0.5024
Epoch 3/50
8368/8368 [==============================] - 30s 4ms/step - loss: 0.6937 - acc: 0.5044 - val_loss: 0.6932 - val_acc: 0.4976
Epoch 4/50
8368/8368 [==============================] - 29s 4ms/step - loss: 0.6860 - acc: 0.5684 - val_loss: 0.7040 - val_acc: 0.5349
Epoch 5/50
8368/8368 [==============================] - 30s 4ms/step - loss: 0.6726 - acc: 0.6048 - val_loss: 0.7058 - val_acc: 0.5311
Epoch 6/50
8368/8368 [==============================] - 31s 4ms/step - loss: 0.6719 - acc: 0.6072 - val_loss: 0.7111 - val_acc: 0.5244
Epoch 7/50
8368/8368 [==============================] - 29s 4ms/step - loss: 0.6726 - acc: 0.6034 - val_loss: 0.7142 - val_acc: 0.5292
Epoch 8